In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import glob
import os


In [28]:
csv_files = glob.glob('oulad_data/*.csv')
dataframes = {os.path.basename(file): pd.read_csv(file) for file in csv_files}

In [29]:
dataframes.keys()

dict_keys(['assessments.csv', 'courses.csv', 'studentAssessment.csv', 'studentInfo.csv', 'studentRegistration.csv', 'studentVle.csv', 'vle.csv'])

In [30]:
for name in list(dataframes.keys()):
    old_name = name
    new_name = name.removesuffix(".csv")
    dataframes.update( {new_name: dataframes.pop(old_name)})

In [31]:
dataframes.keys()

dict_keys(['assessments', 'courses', 'studentAssessment', 'studentInfo', 'studentRegistration', 'studentVle', 'vle'])

In [32]:
dataframes['assessments'].head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1,TMA,30,20
1,AAA,2013J,2,TMA,75,20
2,AAA,2013J,3,TMA,120,20
3,AAA,2013J,4,Exam,230,100
4,AAA,2014B,5,TMA,30,20


In [33]:
dataframes['courses'].head()

,code_module,code_presentation,length
0,AAA,2013J,240
1,AAA,2014B,240
2,BBB,2013J,240
3,BBB,2014B,240
4,CCC,2013J,240


In [34]:
dataframes['studentAssessment'].head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1,9,26,0,29
1,1,10,28,0,55
2,1,12,29,0,76
3,1,16,27,0,24
4,1,19,24,0,19


In [35]:
dataframes['studentInfo'].head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail
1,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass
2,DDD,2013J,2,M,South East Region,Post Graduate Qualification,20-30%,55<=,0,90,N,Fail
3,CCC,2014B,3,F,South East Region,HE Qualification,90-100%,0-35,1,120,N,Fail
4,BBB,2014B,4,M,South Region,A Level or Equivalent,70-80%,35-55,1,60,N,Distinction


In [36]:
dataframes['studentRegistration'].head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,BBB,2014B,1,-51,NaN
1,EEE,2014B,2,-40,NaN
2,DDD,2013J,2,-56,NaN
3,CCC,2014B,3,-50,NaN
4,BBB,2014B,4,-39,NaN


In [37]:
dataframes['studentVle'].head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,BBB,2014B,1,68,57,1
1,BBB,2014B,1,63,73,3
2,BBB,2014B,1,67,176,3
3,BBB,2014B,1,76,53,9
4,BBB,2014B,1,79,193,4


In [38]:
dataframes['vle'].head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,1,AAA,2013J,quiz,4,12
1,2,AAA,2013J,subpage,25,32
2,3,AAA,2013J,page,4,7
3,4,AAA,2013J,glossary,7,11
4,5,AAA,2013J,oucontent,20,21


In [39]:
for name in list(dataframes.keys()):
    print(f'number of rows in {name}: {dataframes[name].shape[0]}')

number of rows in assessments: 40
number of rows in courses: 10
number of rows in studentAssessment: 5392
number of rows in studentInfo: 1507
number of rows in studentRegistration: 1507
number of rows in studentVle: 114130
number of rows in vle: 200


In [40]:
df = dataframes['studentInfo'].merge(dataframes['studentAssessment'], on = 'id_student', how = 'left')

In [41]:
df.head(10)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,id_assessment,date_submitted,is_banked,score
0,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,13.0,29.0,0.0,7.0
1,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,14.0,71.0,0.0,51.0
2,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,15.0,117.0,0.0,35.0
3,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,16.0,225.0,0.0,18.0
4,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,25.0,28.0,0.0,13.0
5,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,26.0,71.0,0.0,13.0
6,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,27.0,116.0,0.0,51.0
7,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,28.0,225.0,0.0,25.0
8,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,37.0,27.0,0.0,73.0
9,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,38.0,69.0,0.0,59.0


In [42]:
df.shape

(9052, 16)

In [43]:
df = df.merge(dataframes['assessments'], on = 'id_assessment', how = 'left')

In [44]:
df.head(10)

,code_module_x,code_presentation_x,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,final_result,id_assessment,date_submitted,is_banked,score,code_module_y,code_presentation_y,assessment_type,date,weight
0,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,Fail,13.0,29.0,0.0,7.0,BBB,2014B,CMA,30.0,30.0
1,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,Fail,14.0,71.0,0.0,51.0,BBB,2014B,TMA,75.0,20.0
2,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,Fail,15.0,117.0,0.0,35.0,BBB,2014B,CMA,120.0,30.0
3,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,Fail,16.0,225.0,0.0,18.0,BBB,2014B,Exam,230.0,100.0
4,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,25.0,28.0,0.0,13.0,DDD,2013J,CMA,30.0,30.0
5,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,26.0,71.0,0.0,13.0,DDD,2013J,CMA,75.0,30.0
6,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,27.0,116.0,0.0,51.0,DDD,2013J,TMA,120.0,20.0
7,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,28.0,225.0,0.0,25.0,DDD,2013J,Exam,230.0,100.0
8,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,37.0,27.0,0.0,73.0,EEE,2014B,TMA,30.0,20.0
9,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,...,Pass,38.0,69.0,0.0,59.0,EEE,2014B,TMA,75.0,20.0


In [45]:
print(df.shape)

(9052, 21)


In [46]:
df.drop(columns = ['code_module_y','code_presentation_y'],axis=1, inplace=True)

In [47]:
df.columns = df.columns.str.replace('_x', '')
df.columns

Index(['code_module', 'code_presentation', 'id_student', 'gender', 'region',
       'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts',
       'studied_credits', 'disability', 'final_result', 'id_assessment',
       'date_submitted', 'is_banked', 'score', 'assessment_type', 'date',
       'weight'],
      dtype='object')

In [48]:
df = df.merge(dataframes['courses'], on= ['code_module','code_presentation'], how = 'left')
df.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,id_assessment,date_submitted,is_banked,score,assessment_type,date,weight,length
0,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,13.0,29.0,0.0,7.0,CMA,30.0,30.0,240
1,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,14.0,71.0,0.0,51.0,TMA,75.0,20.0,240
2,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,15.0,117.0,0.0,35.0,CMA,120.0,30.0,240
3,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,N,Fail,16.0,225.0,0.0,18.0,Exam,230.0,100.0,240
4,EEE,2014B,2,F,North Region,A Level or Equivalent,40-50%,0-35,1,150,N,Pass,25.0,28.0,0.0,13.0,CMA,30.0,30.0,240


In [49]:
df.columns

Index(['code_module', 'code_presentation', 'id_student', 'gender', 'region',
       'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts',
       'studied_credits', 'disability', 'final_result', 'id_assessment',
       'date_submitted', 'is_banked', 'score', 'assessment_type', 'date',
       'weight', 'length'],
      dtype='object')

In [50]:
print(df.shape)

(9052, 20)


In [ ]:
df = df.merge(dataframes['studentVle'], on="id_student", how="left")
df.head()

,code_module_x,code_presentation_x,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,score,assessment_type,date_x,weight,length,code_module_y,code_presentation_y,id_site,date_y,sum_click
0,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,7.0,CMA,30.0,30.0,240,BBB,2014B,68,57,1
1,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,7.0,CMA,30.0,30.0,240,BBB,2014B,63,73,3
2,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,7.0,CMA,30.0,30.0,240,BBB,2014B,67,176,3
3,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,7.0,CMA,30.0,30.0,240,BBB,2014B,76,53,9
4,BBB,2014B,1,M,Wales,A Level or Equivalent,20-30%,55<=,2,90,...,7.0,CMA,30.0,30.0,240,BBB,2014B,79,193,4


In [52]:
df.columns

Index(['code_module_x', 'code_presentation_x', 'id_student', 'gender',
       'region', 'highest_education', 'imd_band', 'age_band',
       'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result',
       'id_assessment', 'date_submitted', 'is_banked', 'score',
       'assessment_type', 'date_x', 'weight', 'length', 'code_module_y',
       'code_presentation_y', 'id_site', 'date_y', 'sum_click'],
      dtype='object')